<a href="https://colab.research.google.com/github/eunicedai/Integrative-Analyze/blob/master/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Homework 2** 

> TaiHsuan Wu | 2524355409

Suppose now that we wish to implement a three interval expansion plan using k = 5, k=10, and finally k=15 in one model.  In this case, the "glide-path" solution according to the paper is one where the decisions made for k = 5 are carried over into the solutions for k = 10, and then, the solutions for k = 10 are carried over into the final solution with k=15.  In other words, you are required to solve one IP model in which three different values k = 5,10,15, are used, and the constraints of this model are such that in such a way that the solutions satisfy the "glide-path" property.  Provide such a formulation, and solve it.  Please verify that you obtain the required property.

In [0]:
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')

In [0]:
!pip install -q pyomo

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from __future__ import division
from pyomo.environ import *

In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

In [0]:
!apt-get install -y -qq glpk-utils

In [0]:
ohio = pd.read_csv('gdrive/My Drive/ISE533/ohio_sheet.csv')

In [6]:
ohio.head()

,County,Population,Map_Label,Adjacent Matrix
0,Williams County,"36,804",1,"2,18,19"
1,Defiance County,"38,165",2,"1, 3,18,17"
2,Paulding County,"18,760",3,"2,4,17"
3,Van Wert County,"28,281",4,"3, 5, 15, 16, 17"
4,Mercer County,"40,959",5,"4,6,15"


In [0]:
country = {}

for i in ohio.index:
  ohio.loc[i]['Adjacent Matrix'].replace(" ", "")
  country[i+1] = [int(x) for x in ohio.loc[i]['Adjacent Matrix'].split(",")]
  country[i+1].append(i+1)

In [0]:
def adj_country1(data, matrix, kvalue):
  model = ConcreteModel()
  model.x = Var(range(88), within=Binary)
  model.y = Var(range(88), within=Binary)
  model.obj = Objective(expr = sum(int(data.loc[i]['Population'].replace(",", ""))*model.y[i] for i in data.index), 
                        sense = maximize)
  model.limits = ConstraintList()
  model.limits.add(sum(model.x[i] for i in range(0,88)) <= kvalue)
  for i in range(0,88):
    model.limits.add(sum(model.x[j] for j in range(0,88) if (j+1) in matrix[i+1]) >= model.y[i])
  
  # solve
  solver = SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model)

  res = pd.DataFrame()
  for i in data.index:
    if model.x[i]() == 1:
      res.loc[i+1, 'Country'] = data.loc[i]['County']
      res.loc[i+1, 'Population'] = data.loc[i]['Population']
      res.loc[i+1, 'Principal'] = 1
  
  print(res)

In [22]:
adj_country1(ohio, country, 5)

            Country Population  Principal
11    Warren County    232,173        1.0
21      Wood County    130,696        1.0
55   Licking County    175,769        1.0
60  Cuyahoga County  1,243,857        1.0
75     Stark County    371,574        1.0


In [0]:
k5=[11,21,55,60,75]

In [0]:
def adj_country2(data, matrix, kvalue):
  model = ConcreteModel()
  model.x = Var(range(88), within=Binary)
  model.y = Var(range(88), within=Binary)
  model.obj = Objective(expr = sum(int(data.loc[i]['Population'].replace(",", ""))*model.y[i] for i in data.index), 
                        sense = maximize)
  model.limits = ConstraintList()
  for i in k5:
    model.limits.add(model.x[i-1] == 1)
  model.limits.add(sum(model.x[i] for i in range(0,88)) <= kvalue)
  for i in range(0,88):
    model.limits.add(sum(model.x[j] for j in range(0,88) if (j+1) in matrix[i+1]) >= model.y[i])
  
  # solve
  solver = SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model)

  res = pd.DataFrame()
  for i in data.index:
    if model.x[i]() == 1:
      res.loc[i+1, 'Country'] = data.loc[i]['County']
      res.loc[i+1, 'Population'] = data.loc[i]['Population']
      res.loc[i+1, 'Principal'] = 1
  
  print(res)

In [25]:
adj_country2(ohio, country, 10)

             Country Population  Principal
11     Warren County    232,173        1.0
15   Auglaize County     45,804        1.0
21       Wood County    130,696        1.0
25  Champaign County     38,754        1.0
41      Huron County     58,504        1.0
51    Jackson County     32,384        1.0
55    Licking County    175,769        1.0
60   Cuyahoga County  1,243,857        1.0
75      Stark County    371,574        1.0
78     Geauga County     94,031        1.0


In [0]:
k10=[11,15,21,25,41,51,55,60,75,78]

In [0]:
def adj_country3(data, matrix, kvalue):
  model = ConcreteModel()
  model.x = Var(range(88), within=Binary)
  model.y = Var(range(88), within=Binary)
  model.obj = Objective(expr = sum(int(data.loc[i]['Population'].replace(",", ""))*model.y[i] for i in data.index), 
                        sense = maximize)
  model.limits = ConstraintList()
  for i in k10:
    model.limits.add(model.x[i-1] == 1)
  model.limits.add(sum(model.x[i] for i in range(0,88)) <= kvalue)
  for i in range(0,88):
    model.limits.add(sum(model.x[j] for j in range(0,88) if (j+1) in matrix[i+1]) >= model.y[i])
  
  # solve
  solver = SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model)

  res = pd.DataFrame()
  for i in data.index:
    if model.x[i]() == 1:
      res.loc[i+1, 'Country'] = data.loc[i]['County']
      res.loc[i+1, 'Population'] = data.loc[i]['Population']
      res.loc[i+1, 'Principal'] = 1
  
  print(res)

In [29]:
adj_country3(ohio, country, 15)

             Country Population  Principal
1    Williams County     36,804        1.0
11     Warren County    232,173        1.0
15   Auglaize County     45,804        1.0
21       Wood County    130,696        1.0
25  Champaign County     38,754        1.0
31   Highland County     43,058        1.0
35     Marion County     65,256        1.0
41      Huron County     58,504        1.0
51    Jackson County     32,384        1.0
55    Licking County    175,769        1.0
60   Cuyahoga County  1,243,857        1.0
68     Athens County     65,818        1.0
75      Stark County    371,574        1.0
78     Geauga County     94,031        1.0
87    Belmont County     67,505        1.0
